# Trabalho final de Sistemas Baseados em Conhecimento
### Tema: Geração de texto com Markov
### Alunos: Caio Lucas da Silva Chacon (20200025769) e Jonas Gabriel Leite de Araújo (20200007608)

### Dado o início do texto em português, gerar automaticamente algo que faça sentido usando cadeias de Markov

In [11]:
from scipy.sparse import dok_matrix
import numpy as np
import re
from random import random 

# Leitura do arquivo e processamento textual para melhor análise

def read_text(filename):
    """"
    Função que lê um arquivo de texto e retorna o conteúdo em minúsculas
    """

    with open(filename, 'r', encoding="utf8") as file:
        content = file.read().lower()

        content = re.sub(r'[\n,\.;:\"\'_\-\t\—]', ' ', content)
        content = re.sub(r'\s+', ' ', content)

    return content

content = read_text('domCasmurro.txt')
print(content)


dom casmurro capítulo primeiro do título uma noite destas vindo da cidade para o engenho novo encontrei no trem da central um rapaz aqui do bairro que eu conheço de vista e de chapéu cumprimentou me sentou se ao pé de mim falou da lua e dos ministros e acabou recitando me versos a viagem era curta e os versos pode ser que não fossem inteiramente maus sucedeu porém que como eu estava cansado fechei os olhos três ou quatro vezes tanto bastou para que ele interrompesse a leitura e metesse os versos no bolso continue disse eu acordando já acabei murmurou ele são muito bonitos vi lhe fazer um gesto para tirá los outra vez do bolso mas não passou do gesto estava amuado no dia seguinte entrou a dizer de mim nomes feios e acabou alcunhando me dom casmurro os vizinhos que não gostam dos meus hábitos reclusos e calados deram curso à alcunha que afinal pegou nem por isso me zanguei contei a anedota aos amigos da cidade e eles por graça chamam me assim alguns em bilhetes dom casmurro domingo vou j

In [12]:
def get_words(corpus):
    """
    Retorna todas as palavras
    """
    corpus_words = corpus.split(' ')
    corpus_words= [word for word in corpus_words if word != '']
    corpus_words # [...'a', 'wyvern', ',', 'two', 'of', 'the', 'thousand'...]
    return corpus_words # 2185920

def get_distinct_words_count(corpus_words):
    """
    Função que retorna a quantidade de palavras distintas e um dicionário com o índice de cada palavra
    """

    distinct_words = list(set(corpus_words))
    word_idx_dict = {word: i for i, word in enumerate(distinct_words)}
    distinct_words_count = len(list(set(corpus_words)))
 
    return distinct_words_count, word_idx_dict # 32663

### Definição da probabilidade inicial

In [21]:
from collections import Counter

corpus_words = get_words(content)

distinct_words = list(set(corpus_words))
distinct_words_count, word_idx_dict = get_distinct_words_count(corpus_words)

counter_corpus = Counter(corpus_words)

# probabilidade de cada palavra dividido pela quantidade de palavras no corpus
distinct_words_prob = {word: counter_corpus[word]/len(corpus_words) for word in distinct_words}

print(distinct_words_count)
print(word_idx_dict)
print(distinct_words_prob)

9129
{'elogia': 0, 'círculos': 1, 'causa': 2, 'aflita': 3, 'bordo!': 4, 'amanhã': 5, 'procuram': 6, 'ofendesse': 7, 'paternal': 8, 'causas': 9, 'escândalo': 10, 'árido': 11, 'ajustado': 12, 'cantos': 13, 'gostam': 14, 'formalidade': 15, 'anunciá': 16, 'carro': 17, 'adversário': 18, 'cansadas': 19, 'impulso': 20, 'vertidas': 21, 'homem?”': 22, 'mentir': 23, 'estipêndio': 24, 'brandura': 25, 'conheço': 26, 'suspendendo': 27, 'encostou': 28, 'sonhara': 29, 'graças': 30, 'dedutiva': 31, 'lavadas': 32, 'indispensável': 33, 'armazém': 34, 'compreendido': 35, 'sábado?': 36, 'definitivamente': 37, 'nelas': 38, 'cantiga': 39, 'vivido': 40, 'experimentar': 41, 'solidão': 42, 'pulmões': 43, 'penhores': 44, 'compreendeis': 45, 'repugnasse': 46, 'alcunha': 47, 'após': 48, 'injustas': 49, 'feliz': 50, 'azuis': 51, 'adivinhar': 52, 'escolha': 53, 'agarrei': 54, 'conviva': 55, 'ódio': 56, 'mundo': 57, 'adorno': 58, 'veleidades': 59, 'mobílias': 60, 'lenço?': 61, 'sucedera': 62, 'ante': 63, 'sua': 64, 

In [30]:
n_grams = 2 # Numero de palavras anteriores para prever a próxima
# Criação de conjuntos de palavras de tamanho n_grams
sets_of_n_words = [ ' '.join(corpus_words[i:i+n_grams]) for i, _ in enumerate(corpus_words[:-n_grams]) ]
len_n_words = len(list(set(sets_of_n_words)))

print(sets_of_n_words[:10])

['dom casmurro', 'casmurro capítulo', 'capítulo primeiro', 'primeiro do', 'do título', 'título uma', 'uma noite', 'noite destas', 'destas vindo', 'vindo da']


### Criação da matriz de transição

In [191]:
# Inicialização de uma matriz esparsa com tamanho len_n_words, distincit_words_count
# para armazenar a contagem de palavras
n_grams_frequency_matrix = dok_matrix((len_n_words, distinct_words_count))

# Criação de um dicionário para mapear os conjuntos de palavras para índices
distinct_sets_of_n_words = list(set(sets_of_n_words))

# Indice pra cada n_gram
n_gram_idx_dict = {word: i for i, word in enumerate(distinct_sets_of_n_words)}

# Preenchimento da matriz com as contagens
for i, word in enumerate(sets_of_n_words[:-n_grams]): # Percorre todos os bigramas
    # Linha da matriz
    n_gram_idx = n_gram_idx_dict[word] # Indice do n_gram
    # Coluna da matriz
    next_word_idx = word_idx_dict[corpus_words[i+n_grams]] # Palavra que aparece logo após o n_grama
    
    # Preenchimento da matriz com a quantidade de vezes que o bigrama ocorre
    n_grams_frequency_matrix[n_gram_idx, next_word_idx] += 1
    
print("(indice_Bigrama, indice_proxima_palavra) = qtd_ocorrencias")
print(n_grams_frequency_matrix)

(indice_Bigrama, indice_proxima_palavra) = qtd_ocorrencias
  (14802, 2666)	1.0
  (34706, 3406)	1.0
  (10605, 1439)	1.0
  (40576, 6407)	1.0
  (1727, 2851)	1.0
  (3500, 9063)	1.0
  (39724, 4413)	1.0
  (19034, 4581)	1.0
  (34384, 5637)	1.0
  (1284, 2816)	1.0
  (14736, 3841)	1.0
  (28093, 7503)	1.0
  (29049, 8186)	2.0
  (28917, 2235)	2.0
  (5524, 1740)	1.0
  (17385, 1512)	1.0
  (37443, 7526)	1.0
  (27683, 5637)	1.0
  (33581, 6583)	2.0
  (42288, 1926)	1.0
  (32104, 5577)	1.0
  (23369, 9108)	1.0
  (13926, 1439)	1.0
  (3686, 4412)	1.0
  (4696, 3301)	1.0
  :	:
  (35294, 153)	1.0
  (523, 8662)	1.0
  (36686, 4884)	1.0
  (18056, 1693)	1.0
  (1649, 153)	1.0
  (35169, 7810)	1.0
  (21984, 7960)	1.0
  (27123, 8528)	1.0
  (14401, 7503)	1.0
  (35780, 7950)	1.0
  (29824, 1504)	1.0
  (26597, 7655)	1.0
  (16027, 2437)	1.0
  (32632, 1693)	1.0
  (26022, 4274)	1.0
  (25215, 4832)	1.0
  (22386, 708)	1.0
  (6205, 4434)	1.0
  (18226, 3497)	1.0
  (27924, 5043)	1.0
  (35709, 1916)	1.0
  (42117, 4536)	1.0
  (15564

### Mudar forma que o weighted choice funciona
Atualmente o que ele faz é pegar o primeiro item do vetor weights_array, que é o vetor com a probabilidade de cada palavra aparecer 

In [372]:
# EXCLUIR ESSA AQUI
weights_array = n_grams_frequency_matrix[n_gram_idx_dict["o engenho"]].toarray().cumsum()
# [0,1/7, 0, 0, 1/7, 0, 0]

sum_of_weights = weights_array.sum() # A soma vai ser sempre igual a 1
# standardization:
np.multiply(weights_array, 1000 / sum_of_weights, weights_array)
weights_array = weights_array.cumsum()
weights_array[2200:2300]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.14505367, 0.29010734, 0.43516101, 0.58021468, 0.72526835,
       0.87032202, 1.01537569, 1.16042936, 1.30548303, 1.4505367 ,
       1.59559037, 1.74064404, 1.88569771, 2.03075138, 2.17580505,
       2.32085872, 2.46591239, 2.61096606, 2.75601973, 2.9010734 ,
       3.04612707, 3.19118074, 3.33623441, 3.48128808, 3.62634175,
       3.77139542, 3.91644909, 4.06150276, 4.20655643, 4.3516101 ,
       4.49666377, 4.64171744, 4.78677111, 4.93182478, 5.07687845,
       5.22193211, 5.36698578, 5.51203945, 5.65709312, 5.80214

In [367]:
def weighted_choice(weights_array, distinct_words):
    """ 
    Função que retorna um elemento aleatório da sequência de 'objetos',
    a probabilidade dos objetos é ponderada de acordo com a porcentagem.
    """
    sum_of_weights = weights_array.sum() # A soma vai ser sempre igual a 1
    # standardization:
    np.multiply(weights_array, 1000 / sum_of_weights, weights_array)
    weights_array = weights_array.cumsum()
    print("Pesos acumulados", weights_array)
    x = random()
    print("Número aleatório", x)

    for i in range(len(weights_array)): # len(weights_array) = len(distinct_words)
        if x < weights_array[i]: # Se o número aleatório for menor que a probabilidade acumulada
            print("i = ", i)
            return distinct_words[i]
        
def get_next_word(word_sequence):
    """
    Função que retorna a próxima palavra após a sequência de palavras
    """
    next_word_vector = n_grams_frequency_matrix[n_gram_idx_dict[word_sequence]] # Linha do bigrama
    next_word_probability = next_word_vector/next_word_vector.sum()

    next_word = weighted_choice(next_word_probability.toarray(), distinct_words)
    
    print("Probabilidade da próxima palavra" , next_word_probability.toarray())
    print("Próxima palavra" , next_word)    
    return next_word
    
def markov_chain(initial_text, chain_length=30):
    """
    Função que gera um texto de acordo com o texto inicial
    """
    current_words = initial_text.split(' ')
    sentence = initial_text

    for _ in range(chain_length):
        sentence += ' '
        next_word = get_next_word(' '.join(current_words))
        sentence += next_word
        current_words = current_words[1:] + [next_word]
    return sentence

In [368]:
markov_chain('o engenho', chain_length=1)

Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.805396844144368
i =  2235
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra novo


'o engenho novo'

In [373]:
markov_chain('capitu não', chain_length=50)

Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.7285910403678096
i =  1262
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra saía
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.6886841697745298
i =  5352
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra sem
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.6766352335289413
i =  7480
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra falar
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.7135694899828526
i =  4832
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra me
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.5306499597649926
i =  1693
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra e
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.19

'capitu não saía sem falar me e a sua ferida uma vez olhou para fora após alguns instantes capítulo xcv o papa a amizade que ela é escreve me muita vez tão involuntária como a fala como tudo quem não desejasse uma queda a algumas adivinhei que traziam as meias muito lavadas'

In [ ]:
markov_chain('agora que expliquei', chain_length=50)

'agora que expliquei o título passo a escrever o livro antes disso porém digamos os motivos que me põem a pena na mão sim nero augusto massinissa e tu grande césar que me incitas a fazer os meus comentários agradeço vos o conselho e vou deitar ao papel as reminiscências que me vierem'